In [1]:
import gurobipy as gp
from gurobipy import GRB
from gurobipy import *
import pandas as pd
import numpy as np

In [2]:
########################################
########### IMPORT DATA ################
########################################

#list of teams that are included in the analysis
Team_list={
0:"Atlanta Falcons",1:"Carolina Panthers",2:"Chicago Bears",3:"Detroit Lions",
4:"Green Bay Packers",5:"Minnesota Vikings",6:"New Orleans Saints",7:"New York Giants",
8:"Philadelphia Eagles",9:"Tampa Bay Buccaneers",10:"Washington Football Team",11:"Baltimore Ravens",
12:"Buffalo Bills",13:"Cincinnati Bengals",14:"Cleveland Browns",15:"Houston Texans",
16:"Indianapolis Colts",17:"Jacksonville Jaguars",18:"Miami Dolphins",19:"New England Patriots",
20:"New York Jets",21:"Pittsburgh Steelers",22:"Tennessee Titans",23:"Dallas Cowboys"}


W=set(list(range(0,13))) #set of weeks, total of 12 weeks
T=set(list(range(0,24)))

#set of teams all, total of 24 teams
 #set of conference
D1=set(list(range(0,12)))  #set of AFC teams
D2=set(list(range(12,24))) #set of NFC teams

#read the distance file
dist=pd.read_csv("distance.csv",index_col=0)

In [7]:
########################################
########### MODEL ######################
########################################

# Create an empty model
m3= gp.Model(name="NFL2b")

# ADD DECISION VARIABLES HERE
dist=pd.read_csv("distance.csv",index_col=0)
games = m3.addVars(W,Team_list,Team_list, vtype = GRB.BINARY, name = "game")
########################################
#### CONSTRAINTS & OBJ FUNCTIONS #######
########################################

# ADD CONSTRAINTS HERE

#Season is 12 weeks long

con1 = m3.addConstrs(gp.quicksum(games[k,i,j] for i in T for j in T if i!=j) == 12 for k in W if k != 8)
    
#Each team would play once per week
    
con2 = m3.addConstrs((gp.quicksum((games[k,i,j] + games[k,j,i] for j in T if j != i)) == 1 for k in W if k != 8 for i in T))
    
#All 12 games that a team plays is against a different opponent

con3 = m3.addConstrs(gp.quicksum(games[k,i,j] + games[k,j,i] for k in W ) <= 1 for i in T for j in T if i != j)
    
#Each team would play at most six home games 

con4 = m3.addConstrs(gp.quicksum(games[k,i,j] for j in T for k in W if j != i) <= 6 for i in T)

#No team would play more than (i) two consecutive games at home and (ii) two consecutive games away

con5 = m3.addConstrs(gp.quicksum(games[k,i,j] + games[k+1,i,j]  + games[k+2,i,j] for j in T) <= 2 for i in T for k in range(0,10))

con6 = m3.addConstrs(gp.quicksum(games[k,i,j] + games[k+1,i,j]  + games[k+2,i,j] for i in T) <= 2 for j in T for k in range(0,10))


# ADD OBJECTIVE FUNCTION HERE

obj = m3.setObjective(gp.quicksum(2*dist.iloc[i,j]*games[k,i,j] for i in T for j in T for k in W), GRB.MINIMIZE)
#optimize the model
m3.optimize()

Gurobi Optimizer version 9.0.3 build v9.0.3rc0 (mac64)
Optimize a model with 1356 rows, 7488 columns and 75960 nonzeros
Model fingerprint: 0xdb9efde7
Variable types: 0 continuous, 7488 integer (7488 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e+01, 4e+03]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+01]
Presolve removed 420 rows and 864 columns
Presolve time: 0.13s
Presolved: 936 rows, 6624 columns, 55752 nonzeros
Variable types: 0 continuous, 6624 integer (6624 binary)

Root relaxation: objective 1.659080e+05, 4874 iterations, 0.71 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0 165908.000    0  162          - 165908.000      -     -    3s
H    0     0                    171934.00000 165908.000  3.50%     -    3s
     0     0 165908.000    0  437 171934.000 165908.000  3.50%     -    7s
H    0     0  

In [34]:
########################################
########### PRINT RESULTS ##############
########################################

# ADD PRINTING HERE:
#loop all 3 dimensions, if statement selecting 14
# print the total distance travelled by all teams
print('\nValue of objective function: %g' % m3.objVal)

# Print optimal values for the decision variables
print('\nDecision variables:')
for v in m3.getVars():
    print('%s = %g' % (v.varName, v.x))


Value of objective function: 165908

Decision variables:
game[0,0,0] = 0
game[0,0,1] = -0
game[0,0,2] = -0
game[0,0,3] = -0
game[0,0,4] = -0
game[0,0,5] = -0
game[0,0,6] = 1
game[0,0,7] = -0
game[0,0,8] = -0
game[0,0,9] = -0
game[0,0,10] = -0
game[0,0,11] = -0
game[0,0,12] = -0
game[0,0,13] = -0
game[0,0,14] = -0
game[0,0,15] = -0
game[0,0,16] = -0
game[0,0,17] = -0
game[0,0,18] = -0
game[0,0,19] = -0
game[0,0,20] = -0
game[0,0,21] = -0
game[0,0,22] = -0
game[0,0,23] = -0
game[0,1,0] = -0
game[0,1,1] = 0
game[0,1,2] = -0
game[0,1,3] = -0
game[0,1,4] = -0
game[0,1,5] = -0
game[0,1,6] = -0
game[0,1,7] = -0
game[0,1,8] = -0
game[0,1,9] = 1
game[0,1,10] = 0
game[0,1,11] = -0
game[0,1,12] = -0
game[0,1,13] = -0
game[0,1,14] = -0
game[0,1,15] = -0
game[0,1,16] = -0
game[0,1,17] = -0
game[0,1,18] = -0
game[0,1,19] = -0
game[0,1,20] = -0
game[0,1,21] = -0
game[0,1,22] = -0
game[0,1,23] = 0
game[0,2,0] = -0
game[0,2,1] = -0
game[0,2,2] = 0
game[0,2,3] = -0
game[0,2,4] = -0
game[0,2,5] = -0
gam

game[2,12,11] = -0
game[2,12,12] = 0
game[2,12,13] = -0
game[2,12,14] = 1
game[2,12,15] = -0
game[2,12,16] = -0
game[2,12,17] = -0
game[2,12,18] = -0
game[2,12,19] = -0
game[2,12,20] = -0
game[2,12,21] = -0
game[2,12,22] = -0
game[2,12,23] = -0
game[2,13,0] = -0
game[2,13,1] = -0
game[2,13,2] = -0
game[2,13,3] = -0
game[2,13,4] = -0
game[2,13,5] = 1
game[2,13,6] = -0
game[2,13,7] = -0
game[2,13,8] = -0
game[2,13,9] = -0
game[2,13,10] = -0
game[2,13,11] = -0
game[2,13,12] = -0
game[2,13,13] = 0
game[2,13,14] = -0
game[2,13,15] = -0
game[2,13,16] = -0
game[2,13,17] = -0
game[2,13,18] = -0
game[2,13,19] = -0
game[2,13,20] = -0
game[2,13,21] = -0
game[2,13,22] = -0
game[2,13,23] = -0
game[2,14,0] = -0
game[2,14,1] = -0
game[2,14,2] = -0
game[2,14,3] = -0
game[2,14,4] = -0
game[2,14,5] = -0
game[2,14,6] = -0
game[2,14,7] = -0
game[2,14,8] = -0
game[2,14,9] = -0
game[2,14,10] = -0
game[2,14,11] = -0
game[2,14,12] = -0
game[2,14,13] = -0
game[2,14,14] = 0
game[2,14,15] = -0
game[2,14,16] = -0

game[5,2,23] = -0
game[5,3,0] = -0
game[5,3,1] = -0
game[5,3,2] = -0
game[5,3,3] = 0
game[5,3,4] = -0
game[5,3,5] = -0
game[5,3,6] = -0
game[5,3,7] = -0
game[5,3,8] = -0
game[5,3,9] = -0
game[5,3,10] = -0
game[5,3,11] = -0
game[5,3,12] = -0
game[5,3,13] = -0
game[5,3,14] = -0
game[5,3,15] = -0
game[5,3,16] = -0
game[5,3,17] = -0
game[5,3,18] = -0
game[5,3,19] = -0
game[5,3,20] = -0
game[5,3,21] = -0
game[5,3,22] = -0
game[5,3,23] = -0
game[5,4,0] = -0
game[5,4,1] = -0
game[5,4,2] = -0
game[5,4,3] = -0
game[5,4,4] = 0
game[5,4,5] = -0
game[5,4,6] = -0
game[5,4,7] = -0
game[5,4,8] = -0
game[5,4,9] = -0
game[5,4,10] = -0
game[5,4,11] = -0
game[5,4,12] = -0
game[5,4,13] = -0
game[5,4,14] = -0
game[5,4,15] = -0
game[5,4,16] = -0
game[5,4,17] = -0
game[5,4,18] = -0
game[5,4,19] = -0
game[5,4,20] = -0
game[5,4,21] = -0
game[5,4,22] = -0
game[5,4,23] = -0
game[5,5,0] = -0
game[5,5,1] = -0
game[5,5,2] = -0
game[5,5,3] = -0
game[5,5,4] = -0
game[5,5,5] = 0
game[5,5,6] = -0
game[5,5,7] = -0
game[

game[8,14,6] = 0
game[8,14,7] = 0
game[8,14,8] = 0
game[8,14,9] = 0
game[8,14,10] = 0
game[8,14,11] = 0
game[8,14,12] = 0
game[8,14,13] = 0
game[8,14,14] = 0
game[8,14,15] = 0
game[8,14,16] = 0
game[8,14,17] = 0
game[8,14,18] = 0
game[8,14,19] = 0
game[8,14,20] = 0
game[8,14,21] = 0
game[8,14,22] = 0
game[8,14,23] = 0
game[8,15,0] = 0
game[8,15,1] = 0
game[8,15,2] = 0
game[8,15,3] = 0
game[8,15,4] = 0
game[8,15,5] = 0
game[8,15,6] = 0
game[8,15,7] = 0
game[8,15,8] = 0
game[8,15,9] = 0
game[8,15,10] = 0
game[8,15,11] = 0
game[8,15,12] = 0
game[8,15,13] = 0
game[8,15,14] = 0
game[8,15,15] = 0
game[8,15,16] = 0
game[8,15,17] = 0
game[8,15,18] = 0
game[8,15,19] = 0
game[8,15,20] = 0
game[8,15,21] = 0
game[8,15,22] = 0
game[8,15,23] = 0
game[8,16,0] = 0
game[8,16,1] = 0
game[8,16,2] = 0
game[8,16,3] = 0
game[8,16,4] = 0
game[8,16,5] = 0
game[8,16,6] = 0
game[8,16,7] = 0
game[8,16,8] = 0
game[8,16,9] = 0
game[8,16,10] = 0
game[8,16,11] = 0
game[8,16,12] = 0
game[8,16,13] = 0
game[8,16,14] = 

game[12,1,13] = -0
game[12,1,14] = -0
game[12,1,15] = -0
game[12,1,16] = -0
game[12,1,17] = -0
game[12,1,18] = -0
game[12,1,19] = -0
game[12,1,20] = -0
game[12,1,21] = -0
game[12,1,22] = -0
game[12,1,23] = -0
game[12,2,0] = -0
game[12,2,1] = -0
game[12,2,2] = 0
game[12,2,3] = -0
game[12,2,4] = -0
game[12,2,5] = -0
game[12,2,6] = -0
game[12,2,7] = -0
game[12,2,8] = -0
game[12,2,9] = -0
game[12,2,10] = -0
game[12,2,11] = -0
game[12,2,12] = -0
game[12,2,13] = -0
game[12,2,14] = 0
game[12,2,15] = -0
game[12,2,16] = -0
game[12,2,17] = -0
game[12,2,18] = -0
game[12,2,19] = 0
game[12,2,20] = -0
game[12,2,21] = -0
game[12,2,22] = -0
game[12,2,23] = -0
game[12,3,0] = -0
game[12,3,1] = -0
game[12,3,2] = -0
game[12,3,3] = 0
game[12,3,4] = 1
game[12,3,5] = -0
game[12,3,6] = -0
game[12,3,7] = -0
game[12,3,8] = -0
game[12,3,9] = -0
game[12,3,10] = -0
game[12,3,11] = -0
game[12,3,12] = -0
game[12,3,13] = -0
game[12,3,14] = -0
game[12,3,15] = -0
game[12,3,16] = -0
game[12,3,17] = -0
game[12,3,18] = -0

In [44]:
m3.printAttr('X')


    Variable            X 
-------------------------
 game[0,0,6]            1 
 game[0,1,9]            1 
game[0,2,23]            1 
game[0,3,19]            1 
 game[0,4,5]            1 
game[0,7,17]            1 
game[0,11,10]            1 
game[0,16,15]            1 
game[0,18,8]            1 
game[0,20,12]            1 
game[0,21,14]            1 
game[0,22,13]            1 
game[1,1,15]            1 
 game[1,3,7]            1 
 game[1,4,2]            1 
game[1,5,12]            1 
game[1,8,17]            1 
game[1,10,21]            1 
game[1,14,19]            1 
game[1,16,0]            1 
game[1,18,11]            1 
game[1,20,9]            1 
game[1,22,6]            1 
game[1,23,13]            1 
game[2,0,23]            1 
 game[2,2,3]            1 
game[2,6,16]            1 
game[2,9,18]            1 
game[2,10,1]            1 
game[2,11,8]            1 
game[2,12,14]            1 
game[2,13,5]            1 
game[2,15,4]            1 
game[2,17,22]            1 
game[2,19,20]    

In [38]:
#games[k=week,i=home,j=away]
#a = home, b=away

distance1= pd.DataFrame(columns = W, index =T)
for i in T:
    for k in W:
        for j in T:
            a=m3.getVarByName("game[%s,%s,%s]"%(k,i,j)).x
            b=m3.getVarByName("game[%s,%s,%s]"%(k,j,i)).x
            if(a==1.0 and b!=1.0):
                distance1.iloc[i,k]=0
                distance1.iloc[j,k]=2*dist.iloc[i,j]
            elif(b==1.0 and a!=1.0):
                distance1.iloc[j,k]=0
                distance1.iloc[i,k]=2*dist.iloc[i,j]
#replace.nan
distance1.fillna(0,inplace=True)                
distance1



,0,1,2,3,4,5,6,7,8,9,10,11,12
0,0,1068,0,0,692,1292,0,0,0,0,2226,500,922
1,0,0,814,1250,0,1430,0,0,0,490,0,766,1648
2,0,414,0,944,0,920,0,1056,0,368,0,0,688
3,0,0,568,1166,0,526,540,0,0,340,0,1226,0
4,0,0,2574,0,0,788,2346,0,0,1366,0,1014,978
5,558,0,1408,0,818,0,0,1382,0,2352,0,1880,0
6,940,1064,0,0,2260,0,2398,0,0,1702,0,1850,0
7,0,1226,728,0,0,0,0,0,0,0,1250,408,710
8,2358,0,202,0,192,0,0,1074,0,2084,0,724,0
9,1160,2258,0,0,1962,0,912,0,0,0,1314,1822,0


In [42]:
team_dist=distance1.sum(axis=1)
team_dist.sort_values()

21     3498
20     3656
7      4322
3      4366
2      4390
14     4782
10     5152
13     5412
12     5950
1      6398
16     6488
8      6634
0      6700
11     6836
19     7722
17     8292
5      8398
22     9042
18     9064
4      9066
23     9198
9      9428
6     10214
15    10900
dtype: int64